In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors

In [ ]:
def add_mathes(df, radius):
    dfs = []

    for country, country_df in tqdm(df.groupby("country")):
        country_df = country_df.reset_index(drop=True)
        
        knn = NearestNeighbors(radius=radius, n_jobs=-1)
        knn.fit(country_df[['latitude','longitude']])
        nears = knn.radius_neighbors(country_df[['latitude','longitude']], return_distance=False)
        
        for j, n in enumerate(nears):
            country_df.loc[j, "matches"] = " ".join([country_df.loc[i, "id"] for i in n])
        country_df['matches'] = country_df["id"]

        dfs.append(country_df)

    null_idx = pd.isnull(df['country'])
    null_df = df[null_idx]
    null_df['matches'] = null_df["id"]
    dfs.append(null_df)
    
    df = pd.concat(dfs).reset_index(drop=True)
    return df

In [ ]:
test = pd.read_csv('/kaggle/input/foursquare-location-matching/test.csv')
test['matches'] = ''
test = add_mathes(test, 0.01)

In [ ]:
submit = pd.read_csv("/kaggle/input/foursquare-location-matching/sample_submission.csv")
submit = submit.drop(columns="matches")
submit = submit.merge(test[["id", "matches"]], on="id")
submit.to_csv("submission.csv", index=False)

submit.head()